# ASL Alphabet Classifier Test #1

#### Datasource: 
<a href="https://www.kaggle.com/grassknoted/asl-alphabet">https://www.kaggle.com/grassknoted/asl-alphabet</a>

<br>

This is a prototype, playing with FastAI using Resnet34 to classify American Sign Language alphabet.  It's basically Notebook #2 from the MOOC on a "clean" dataset.  We get great results ... but the data is really contrived.  It's highly likely the model will be overfitted, however it's a good test of the library.
<br>
<br>
<br>

In [8]:
# !pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *
import fastai

plt.style.use('classic')


In [9]:
path = '../data/external/Training Set'

<br>

### Create the DataBlock

In [10]:
signs = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label)

<br>

## Resize and add basic image augmentation

In [12]:
# signs = signs.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=4))
signs = signs.new(item_tfms=Resize(128, method=ResizeMethod.Squish))

# signs.apply_tfms(Image.flip)


AttributeError: module 'PIL.Image' has no attribute 'flip_lr'

<br>

### Load the data by path

In [ ]:
dls = signs.dataloaders(path)

<br>

### Verify the Training and Validation Batches

In [ ]:
dls.train.show_batch(max_n=4, nrows=1)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

<br>

### Define the model (Resnet34) and fit

In [ ]:
learn = cnn_learner(dls, resnet101 , metrics=error_rate)
learn.fine_tune(120)

In [ ]:
learn = cnn_learner(dls, resnet18 , metrics=error_rate)
learn.fine_tune(4)

In [ ]:
# The first time, we tuned it just once in order to not over-fit.  
# Now we're going to tune it 10 times to see if we can REALLY fit it.
learn.fine_tune(10)

In [ ]:
# The first time, we tuned it just once in order to not over-fit.  
# Now we're going to tune it 10 times to see if we can REALLY fit it.
learn.fine_tune(10)

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.recorder.plot_lr_find()

In [ ]:
learn.fit_one_cycle(4, 10e-4)

In [ ]:
learn.recorder.plot_lr_find()

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(3)

<br>

### Visualize with a confusion Matrix

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(8,8))

<br>

### Show the top 5 most error prone images

In [ ]:
interp.plot_top_losses(5, nrows=1)

<br>
<br>
<hr>
<br>
<br>

## Persist the Model

<br>
<br>

In [ ]:
learn.export('../models/export-resnet101-tune33.pkl')
path = Path('../models')
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
learn_inf.predict('../data/raw/darktable_exported/D-frank.jpg')

In [ ]:
learn_inf.predict('../data/raw/darktable_exported/aA2.jpg')

In [ ]:
learn_inf.dls.vocab

In [ ]:
pred,pred_idx,probs = learn_inf.predict('../data/raw/darktable_exported/aA2.jpg')

lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [ ]:
pred,pred_idx,probs = learn_inf.predict('../data/raw/darktable_exported/D-frank.jpg')

lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

<br>
<br>

## Make a Notebook App

<br>

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
# img = PILImage.create(btn_upload.data[-1])


In [ ]:
out_pl = widgets.Output()
out_pl.clear_output()
# with out_pl: display(img.to_thumb(200,200))
# out_pl

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(200,200))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
    
btn_run.on_click(on_click_classify)

In [ ]:
VBox([widgets.Label('Select an image: '),
    btn_upload, btn_run, out_pl, lbl_pred])

In [ ]:
import fastbook
?fastbook


In [ ]:
import pandas as pd
pd.DataFrame()


In [ ]:
?DataBlock